<a href="https://colab.research.google.com/github/Calex192/Desafio-Telecom-X-parte-2.-Alura-Latam./blob/main/Datos_tratados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extracción del Archivo Tratado.**

In [11]:
import pandas as pd

url = "https://raw.githubusercontent.com/Calex192/Desafio-Telecom-X-parte-2.-Alura-Latam./refs/heads/main/datos_tratados.csv"
df = pd.read_csv(url)
df.head()


,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges.monthly,charges.total,cuentas_diarias
0,0,0,0,1,1,9,1,no,dsl,no,...,no,yes,yes,no,one year,1,mailed check,65.6,593.30,2.186667
1,0,1,0,0,0,9,1,yes,dsl,no,...,no,no,no,yes,month-to-month,0,mailed check,59.9,542.40,1.996667
2,1,1,0,0,0,4,1,no,fiber optic,no,...,yes,no,no,no,month-to-month,1,electronic check,73.9,280.85,2.463333
3,1,1,1,1,0,13,1,no,fiber optic,no,...,yes,no,yes,yes,month-to-month,1,electronic check,98.0,1237.85,3.266667
4,1,0,1,1,0,3,1,no,fiber optic,no,...,no,yes,yes,no,month-to-month,1,mailed check,83.9,267.40,2.796667


**Eliminación de Columnas Irrelevantes.**

In [12]:
import pandas as pd
import numpy as np

# 0) Trabajamos sobre una copia
dfc = df.copy()

# 1) IDs evidentes y columnas de índice/ruido
id_like_exact = ['customerid','customer_id','id','level_0','index','unnamed: 0']
to_drop = [c for c in dfc.columns if c.lower() in id_like_exact]

# 1.1) Dummies creados por error a partir de IDs
id_like_prefixes = ['customerid_', 'customer_id_']
to_drop += [c for c in dfc.columns if any(c.lower().startswith(p) for p in id_like_prefixes)]

# 2) Objetos con cardinalidad altísima (probable ID/Texto irrelevante)
hi_card_obj = [
    c for c in dfc.select_dtypes(include='object').columns
    if dfc[c].nunique(dropna=False) > 0.5 * len(dfc)
]
to_drop += hi_card_obj

# 3) Columnas casi constantes (>=98% el mismo valor)
near_constant = []
for c in dfc.columns:
    vc = dfc[c].value_counts(dropna=False, normalize=True)
    if not vc.empty and vc.iloc[0] >= 0.98:
        near_constant.append(c)

# No borres el target si aparece aquí
near_constant = [c for c in near_constant if c != 'churn']
to_drop += near_constant

# 4) Redundancia financiera: preferimos tenure + charges.monthly; quitamos charges.total si están las tres
if {'tenure','charges.monthly','charges.total'}.issubset(set(dfc.columns)):
    to_drop.append('charges.total')

# 5) Dummies de "no internet service" (opcional) si ya tienes la base en internetservice
no_int_cols = [c for c in dfc.columns if str(c).lower().endswith('no_internetservice')]
# Si sientes que duplican la información, descomenta la siguiente línea:
# to_drop += no_int_cols

# Consolidamos lista y eliminamos
to_drop = sorted(set(to_drop))
print("Se eliminarán estas columnas (si existen):")
print(to_drop)

dfc = dfc.drop(columns=to_drop, errors='ignore')
print("\nShape original:", df.shape, "→ Shape tras limpieza:", dfc.shape)


Se eliminarán estas columnas (si existen):
['charges.total']

Shape original: (7032, 21) → Shape tras limpieza: (7032, 20)


In [13]:
# Definimos el target
assert 'churn' in dfc.columns, "No encuentro la columna churn en el dataframe!"
y = dfc['churn']

# Variables explicativas (X) aún sin codificar
X = dfc.drop(columns=['churn'])

# Detectamos categóricas "reales" (tipo object o category)
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
print("Categóricas a codificar:", cat_cols)

# One-hot encoding con primera categoría como base para evitar colinealidad perfecta
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

print("Shape final de X:", X.shape)


Categóricas a codificar: ['multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paymentmethod']
Shape final de X: (7032, 30)


In [14]:
# Reduce memoria
for c in X.select_dtypes(include=['float64']).columns:
    X[c] = X[c].astype('float32')
for c in X.select_dtypes(include=['int64']).columns:
    X[c] = X[c].astype('int32')


In [15]:
# Correlaciones altas entre numéricas (para decidir cuál quitar)
num_cols = X.select_dtypes(include=np.number).columns
corr = X[num_cols].corr().abs()

# Tomamos pares con correlación > 0.95
high_pairs = [
    (i,j,corr.loc[i,j]) for i in num_cols for j in num_cols
    if i<j and corr.loc[i,j] > 0.95
]
high_pairs[:10]  # échales un vistazo

# Puedes decidir manualmente qué columna de cada par dejar y cuál quitar.


[('charges.monthly', 'cuentas_diarias', np.float64(0.9999999999999971))]